### Import Library

In [ ]:
import cv2
import pandas as pd
import shutil
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2, InceptionV3, DenseNet121, InceptionV3, MobileNetV3Large, Xception, VGG16, ResNet50, EfficientNetV2B0
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

import os
os.environ['TF_DETERMINISTIC_OPS'] = '1'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Jenis

## Data Preparation: Jenis

In [ ]:
# Load the train.csv file
csv_path = "/content/drive/MyDrive/HOLOGY/dataset/train.csv"
train_df = pd.read_csv(csv_path)

# Define the root folder for your images
train_image_folder = "/content/drive/MyDrive/HOLOGY/dataset/train"
train_split_folder = "/content/drive/MyDrive/HOLOGY/dataset/train_split_2"
val_split_folder = "/content/drive/MyDrive/HOLOGY/dataset/val_split_2"
test_split_folder = "/content/drive/MyDrive/HOLOGY/dataset/test_split_2"

# Tentukan proporsi pembagian
train_size = 0.8
val_size = 0.1
test_size = 0.1
random_state = 42  # Set random_state untuk memastikan pembagian konsisten

# Bagi train_df menjadi train, val, dan test, dengan random_state
train_split_df, temp_df = train_test_split(train_df, test_size=(val_size + test_size), random_state=random_state)
val_split_df, test_split_df = train_test_split(temp_df, test_size=(test_size / (val_size + test_size)), random_state=random_state)

# Mapping jenis ke bentuk deskriptif
jenis_mapping = {0: 'Kaos', 1: 'Hoodie'}

The code above loads product data, defines directories for image storage, splits the dataset into training, validation, and testing sets, and maps product types to descriptive names. This is the preparation phase for the dataset before using it for training and evaluating a machine learning model.

In [ ]:
# Fungsi untuk mendapatkan path gambar
def get_image_path(image_id, folder):
    jpg_path = os.path.join(folder, f"{image_id}.jpg")
    png_path = os.path.join(folder, f"{image_id}.png")
    if os.path.exists(jpg_path):
        return jpg_path
    elif os.path.exists(png_path):
        return png_path
    else:
        return None

# Menambahkan kolom path gambar ke DataFrame
train_split_df['image_path'] = train_split_df['id'].apply(lambda x: get_image_path(x, train_image_folder))
val_split_df['image_path'] = val_split_df['id'].apply(lambda x: get_image_path(x, train_image_folder))
test_split_df['image_path'] = test_split_df['id'].apply(lambda x: get_image_path(x, train_image_folder))

This code defines a function `get_image_path()` that takes an image ID and a folder path, and returns the full file path for the image, checking if the image is in either `.jpg` or `.png` format.

Next, it adds a new column `image_path` to the DataFrames (`train_split_df`, `val_split_df`, `test_split_df`) by applying the `get_image_path()` function to each image ID in the `id` column. This column will contain the correct file path for each image in the respective dataset split.

In [ ]:
# Fungsi untuk memindahkan gambar ke folder berdasarkan jenis pakaian
def move_images(split_df, split_folder):
    for index, row in split_df.iterrows():
        image_id = row['id']
        jenis = row['jenis']  # Label jenis pakaian (0 atau 1)
        jenis_label = jenis_mapping[jenis]  # Dapatkan label deskriptif untuk jenis

        # Buat folder berdasarkan jenis jika belum ada
        class_folder = os.path.join(split_folder, jenis_label)
        os.makedirs(class_folder, exist_ok=True)

        # Pindahkan gambar ke folder jenis
        image_path = row['image_path']
        if image_path:  # Cek jika path gambar valid
            dst_path = os.path.join(class_folder, os.path.basename(image_path))
            shutil.copy(image_path, dst_path)

# Pindahkan gambar untuk train_split, val_split, dan test_split
move_images(train_split_df, train_split_folder)
move_images(val_split_df, val_split_folder)
move_images(test_split_df, test_split_folder)

print("Gambar berhasil dipindahkan ke folder train_split, val_split, dan test_split berdasarkan jenis pakaian (Kaos/Hoodie).")

The `move_images()` function moves images into folders based on their clothing type (T-shirt or Hoodie). It creates a folder for each type, then copies the images from their original location into the corresponding folder. This process is applied to the training, validation, and test datasets.

In [ ]:
# Folder root untuk train_split, val_split, dan test_split
train_split_folder = "/content/drive/MyDrive/HOLOGY/dataset/train_split_2"
val_split_folder = "/content/drive/MyDrive/HOLOGY/dataset/val_split_2"
test_split_folder = "/content/drive/MyDrive/HOLOGY/dataset/test_split_2"

# Fungsi untuk menghitung jumlah gambar di setiap folder dan subfolder
def count_images_in_folder(folder_path):
    folder_image_count = {}
    total_images = 0

    for root, dirs, files in os.walk(folder_path):
        # Hanya hitung file dengan ekstensi .jpg dan .png
        image_files = [f for f in files if f.endswith('.jpg') or f.endswith('.png')]
        folder_image_count[root] = len(image_files)
        total_images += len(image_files)  # Hitung total gambar

    return folder_image_count, total_images

# Hitung jumlah gambar di setiap folder dan total gambar
train_image_count, total_train_images = count_images_in_folder(train_split_folder)
val_image_count, total_val_images = count_images_in_folder(val_split_folder)
test_image_count, total_test_images = count_images_in_folder(test_split_folder)

# Menampilkan hasil untuk setiap folder dan subfolder
print("Jumlah gambar di setiap folder train_split:")
for folder, count in train_image_count.items():
    print(f"{folder}: {count} gambar")
print(f"Total gambar di train_split: {total_train_images} gambar\n")

print("Jumlah gambar di setiap folder val_split:")
for folder, count in val_image_count.items():
    print(f"{folder}: {count} gambar")
print(f"Total gambar di val_split: {total_val_images} gambar\n")

print("Jumlah gambar di setiap folder test_split:")
for folder, count in test_image_count.items():
    print(f"{folder}: {count} gambar")
print(f"Total gambar di test_split: {total_test_images} gambar\n")

Jumlah gambar di setiap folder train_split:
/content/drive/MyDrive/HOLOGY/dataset/train_split_2: 0 gambar
/content/drive/MyDrive/HOLOGY/dataset/train_split_2/Hoodie: 246 gambar
/content/drive/MyDrive/HOLOGY/dataset/train_split_2/Kaos: 375 gambar
Total gambar di train_split: 621 gambar

Jumlah gambar di setiap folder val_split:
/content/drive/MyDrive/HOLOGY/dataset/val_split_2: 0 gambar
/content/drive/MyDrive/HOLOGY/dataset/val_split_2/Kaos: 52 gambar
/content/drive/MyDrive/HOLOGY/dataset/val_split_2/Hoodie: 26 gambar
Total gambar di val_split: 78 gambar

Jumlah gambar di setiap folder test_split:
/content/drive/MyDrive/HOLOGY/dataset/test_split_2: 0 gambar
/content/drive/MyDrive/HOLOGY/dataset/test_split_2/Hoodie: 29 gambar
/content/drive/MyDrive/HOLOGY/dataset/test_split_2/Kaos: 49 gambar
Total gambar di test_split: 78 gambar



The code defines a function `count_images_in_folder()` that counts the number of `.jpg` and `.png` images in each folder and subfolder within a given directory. It then prints the number of images in each folder for the training, validation, and test splits, as well as the total number of images in each split.

In [ ]:
img_size = (224,224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    fill_mode='nearest'
)

# Validation dan Test tidak memerlukan augmentasi, hanya rescaling
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Membuat generator untuk train dataset
train_generator = train_datagen.flow_from_directory(
    train_split_folder,
    target_size=img_size,  # Sesuaikan dengan ukuran input model Anda
    batch_size=batch_size,
    class_mode='categorical',  # Pilih categorical untuk multi-class classification
    shuffle=True,  # Mengacak data pada setiap epoch
    seed=42  # Seed untuk pengacakan dan augmentasi konsisten
)

# Membuat generator untuk validation dataset
val_generator = val_test_datagen.flow_from_directory(
    val_split_folder,
    target_size=img_size,  # Sesuaikan dengan ukuran input model Anda
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,  # Mengacak data pada setiap epoch
    seed=42  # Seed untuk pengacakan dan augmentasi konsisten
)

# Membuat generator untuk test dataset
test_generator = val_test_datagen.flow_from_directory(
    test_split_folder,
    target_size=img_size,  # Sesuaikan dengan ukuran input model Anda
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Biasanya, test data tidak di-shuffle
)

print("Train, validation, dan test generator berhasil dibuat.")

Found 621 images belonging to 2 classes.
Found 78 images belonging to 2 classes.
Found 78 images belonging to 2 classes.
Train, validation, dan test generator berhasil dibuat.


The code sets up image data generators for training, validation, and test datasets.

It resizes images to 224x224 pixels and normalizes pixel values by scaling them to a range of 0-1. The `train_datagen` applies data augmentation (with the `fill_mode='nearest'` option) to the training set, while the `val_test_datagen` only rescales the images for the validation and test sets.

The generators `train_generator`, `val_generator`, and `test_generator` are created using `flow_from_directory()` to load images from the respective folders, process them, and prepare them for training the model. For the training and validation sets, the data is shuffled, while the test set is not.

## Training Model: Jenis

Regarding the prediction task: To predict the 'jenis' (clothing type) first, the model would classify whether an image is a Kaos (T-shirt) or a Hoodie using the first dense output (2 classes). Once the clothing type is determined, a separate model or a second step can be used to predict the color (Red, Yellow, Blue, Black, White). This approach ensures that the jenis classification comes first before color classification.

### MobileNetV2

In [ ]:
# Definisikan input layer secara eksplisit untuk menetapkan input_shape
input_shape = (224, 224, 3)

# Membuat model base menggunakan MobileNetV2
base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')
base_model.trainable = False

# Bangun model sequential dengan nama variabel yang berbeda
model_kaos_hoodie = models.Sequential([
    layers.InputLayer(input_shape=input_shape),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dense(2, activation='softmax')  # 2 kelas: Kaos dan Hoodie
])

# Kompilasi model
model_kaos_hoodie.compile(
    loss='categorical_crossentropy',  # categorical_crossentropy untuk klasifikasi multi-kelas
    optimizer=Adam(learning_rate=0.0001),  # Optimizer Adam dengan learning rate kecil
    metrics=['accuracy']
)

# Tampilkan ringkasan model
model_kaos_hoodie.summary()

# Callbacks untuk penyimpanan model terbaik dan early stopping
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("best_transfer_learning_model.keras", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

# Training model menggunakan generator
history = model_kaos_hoodie.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,  # Jumlah epoch
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    callbacks=[checkpoint_cb, early_stopping_cb]
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_3           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 512)                 │         655,872 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 2)                   │           1,026 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,914,882 (11.12 MB)

 Trainable params: 656,898 (2.51 MB)

...

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


19/19 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.8280 - loss: 0.4431 - val_accuracy: 0.8906 - val_loss: 0.2052
Epoch 2/20
 1/19 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.9688 - loss: 0.1257

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 69ms/step - accuracy: 0.9688 - loss: 0.1257 - val_accuracy: 0.9286 - val_loss: 0.1399
Epoch 3/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 37s 1s/step - accuracy: 0.9660 - loss: 0.1235 - val_accuracy: 0.9219 - val_loss: 0.1234
Epoch 4/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - accuracy: 0.8750 - loss: 0.3159 - val_accuracy: 1.0000 - val_loss: 0.0660
Epoch 5/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - accuracy: 0.9676 - loss: 0.0943 - val_accuracy: 0.9531 - val_loss: 0.1387
Epoch 6/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - accuracy: 0.8750 - loss: 0.1604 - val_accuracy: 1.0000 - val_loss: 0.0933
Epoch 7/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 37s 1s/step - accuracy: 0.9875 - loss: 0.0689 - val_accuracy: 0.9688 - val_loss: 0.0760
Epoch 8/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - accuracy: 1.0000 - loss: 0.0420 - val_accuracy: 1.0000 - val_loss: 0.0829
Epoch 9/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.9943 - loss: 0.0533 - val_accuracy: 0.9844 - val_loss: 0.0850

In [ ]:
# Evaluasi pada test set
test_loss, test_acc = model_kaos_hoodie.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test Accuracy: {test_acc:.2f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - accuracy: 0.9583 - loss: 0.1426
Test Accuracy: 0.97


The test accuracy of **97%** means that the model correctly predicted the clothing type (T-shirt or Hoodie) for 97% of the test images. This indicates that the model has a high level of performance and generalizes well to unseen data from the test set.

## Inference Model: Jenis

In [ ]:
# Path folder test
test_folder = "/content/drive/MyDrive/HOLOGY/dataset/test"

# Membuat list untuk menyimpan hasil prediksi
image_ids = []
predictions = []

# Looping untuk memproses gambar dari 778 hingga 1111 secara berurutan
for img_id in range(778, 1112):
    # Cek apakah file .jpg atau .png ada
    img_jpg = os.path.join(test_folder, f"{img_id}.jpg")
    img_png = os.path.join(test_folder, f"{img_id}.png")

    if os.path.exists(img_jpg):
        img_path = img_jpg
    elif os.path.exists(img_png):
        img_path = img_png
    else:
        # Jika tidak ditemukan, lanjutkan ke gambar berikutnya
        print(f"Gambar dengan id {img_id} tidak ditemukan.")
        continue

    # Load dan preproses gambar
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Tambahkan dimensi batch
    img_array = img_array / 255.0  # Normalisasi

    # Prediksi menggunakan model yang telah dilatih
    pred = model_kaos_hoodie.predict(img_array)
    pred_class = np.argmax(pred, axis=1)[0]  # Mendapatkan indeks kelas (0 atau 1)

    # Menyimpan id dan hasil prediksi
    image_ids.append(img_id)
    predictions.append(pred_class)  # Simpan 0 (Kaos) atau 1 (Hoodie) sebagai prediksi

# Membuat DataFrame untuk hasil prediksi
final_df = pd.DataFrame({
    "id": image_ids,
    "jenis": predictions  # Hasil prediksi hanya berupa 0 (Kaos) atau 1 (Hoodie)
})

# Menampilkan DataFrame hasil prediksi
print(final_df)

       id  jenis
0     778      0
1     779      1
2     780      1
3     781      0
4     782      1
..    ...    ...
329  1107      1
330  1108      0
331  1109      0
332  1110      1
333  1111      1

[334 rows x 2 columns]


This code loops through image IDs from 778 to 1111 in the `test` folder, checking for each image's existence in either `.jpg` or `.png` format. It then loads and preprocesses each image, resizing it to 224x224 pixels, normalizing the pixel values, and preparing it for prediction. The model predicts whether the image is a T-shirt (0) or Hoodie (1), and the results are stored in a DataFrame with image IDs and their corresponding predictions. The final DataFrame with these predictions is then printed.

In [ ]:
print(final_df[['id', 'jenis']].head(25))

     id  jenis
0   778      0
1   779      1
2   780      1
3   781      0
4   782      1
5   783      0
6   784      1
7   785      1
8   786      1
9   787      0
10  788      0
11  789      0
12  790      0
13  791      0
14  792      0
15  793      1
16  794      0
17  795      0
18  796      0
19  797      0
20  798      0
21  799      0
22  800      1
23  801      1
24  802      0


# Warna

# Data Preparation: Warna

In [ ]:
# Load the train.csv file
csv_path = "/content/drive/MyDrive/HOLOGY/dataset/train.csv"
train_df = pd.read_csv(csv_path)

# Define the root folder for your images
train_image_folder = "/content/drive/MyDrive/HOLOGY/dataset/train"
train_split_folder = "/content/drive/MyDrive/HOLOGY/dataset/train_split_warna"
val_split_folder = "/content/drive/MyDrive/HOLOGY/dataset/val_split_warna"
test_split_folder = "/content/drive/MyDrive/HOLOGY/dataset/test_split_warna"

# Tentukan proporsi pembagian
train_size = 0.8
val_size = 0.1
test_size = 0.1
random_state = 42  # Set random_state untuk memastikan pembagian konsisten

# Bagi train_df menjadi train, val, dan test, dengan random_state
train_split_df, temp_df = train_test_split(train_df, test_size=(val_size + test_size), random_state=random_state)
val_split_df, test_split_df = train_test_split(temp_df, test_size=(test_size / (val_size + test_size)), random_state=random_state)

# Mapping warna ke bentuk deskriptif
warna_mapping = {0: 'Merah', 1: 'Kuning', 2: 'Biru', 3: 'Hitam', 4: 'Putih'}

This code loads the `train.csv` file, sets up folder paths for images, and splits the dataset into training, validation, and test sets. It also defines the proportion for each split (80% train, 10% validation, 10% test) and ensures consistent splitting using a fixed random state. The color labels (0-4) are mapped to their descriptive names (Red, Yellow, Blue, Black, White).

In [ ]:
# Fungsi untuk mendapatkan path gambar
def get_image_path(image_id, folder):
    jpg_path = os.path.join(folder, f"{image_id}.jpg")
    png_path = os.path.join(folder, f"{image_id}.png")
    if os.path.exists(jpg_path):
        return jpg_path
    elif os.path.exists(png_path):
        return png_path
    else:
        return None

# Menambahkan kolom path gambar ke DataFrame
train_split_df['image_path'] = train_split_df['id'].apply(lambda x: get_image_path(x, train_image_folder))
val_split_df['image_path'] = val_split_df['id'].apply(lambda x: get_image_path(x, train_image_folder))
test_split_df['image_path'] = test_split_df['id'].apply(lambda x: get_image_path(x, train_image_folder))

This code defines the `get_image_path()` function, which takes an image ID and folder path, and checks if the image exists as a `.jpg` or `.png` file. It returns the correct file path if found, or `None` if not.

Then, it adds a new column `image_path` to the `train_split_df`, `val_split_df`, and `test_split_df` DataFrames by applying this function to each image ID in the `id` column. This new column stores the file paths of the images.

In [ ]:
# Fungsi untuk memindahkan gambar ke folder berdasarkan warna pakaian
def move_images_by_warna(split_df, split_folder):
    for index, row in split_df.iterrows():
        image_id = row['id']
        warna = row['warna']  # Label warna pakaian (0 hingga 4)
        warna_label = warna_mapping[warna]  # Dapatkan label deskriptif untuk warna

        # Buat folder berdasarkan warna jika belum ada
        class_folder = os.path.join(split_folder, warna_label)
        os.makedirs(class_folder, exist_ok=True)

        # Pindahkan gambar ke folder warna
        image_path = row['image_path']
        if image_path:  # Cek jika path gambar valid
            dst_path = os.path.join(class_folder, os.path.basename(image_path))
            shutil.copy(image_path, dst_path)

# Pindahkan gambar untuk train_split, val_split, dan test_split berdasarkan warna
move_images_by_warna(train_split_df, train_split_folder)
move_images_by_warna(val_split_df, val_split_folder)
move_images_by_warna(test_split_df, test_split_folder)

print("Gambar berhasil dipindahkan ke folder train_split, val_split, dan test_split berdasarkan warna pakaian (Merah, Kuning, Biru, Hitam, Putih).")

Gambar berhasil dipindahkan ke folder train_split, val_split, dan test_split berdasarkan warna pakaian (Merah, Kuning, Biru, Hitam, Putih).


The `move_images_by_warna()` function moves images into folders based on their color labels (Red, Yellow, Blue, Black, White). It iterates through the dataset, checks if the image exists, creates folders for each color, and copies the images into the appropriate folder. The function is applied to the train, validation, and test datasets, and a confirmation message is printed once the images are moved.

In [ ]:
# Folder root untuk train_split, val_split, dan test_split
train_split_folder = "/content/drive/MyDrive/HOLOGY/dataset/train_split_warna"
val_split_folder = "/content/drive/MyDrive/HOLOGY/dataset/val_split_warna"
test_split_folder = "/content/drive/MyDrive/HOLOGY/dataset/test_split_warna"

# Fungsi untuk menghitung jumlah gambar di setiap folder dan subfolder
def count_images_in_folder(folder_path):
    folder_image_count = {}
    total_images = 0

    for root, dirs, files in os.walk(folder_path):
        # Hanya hitung file dengan ekstensi .jpg dan .png
        image_files = [f for f in files if f.endswith('.jpg') or f.endswith('.png')]
        folder_image_count[root] = len(image_files)
        total_images += len(image_files)  # Hitung total gambar

    return folder_image_count, total_images

# Hitung jumlah gambar di setiap folder dan total gambar
train_image_count, total_train_images = count_images_in_folder(train_split_folder)
val_image_count, total_val_images = count_images_in_folder(val_split_folder)
test_image_count, total_test_images = count_images_in_folder(test_split_folder)

# Menampilkan hasil untuk setiap folder dan subfolder
print("Jumlah gambar di setiap folder train_split:")
for folder, count in train_image_count.items():
    print(f"{folder}: {count} gambar")
print(f"Total gambar di train_split: {total_train_images} gambar\n")

print("Jumlah gambar di setiap folder val_split:")
for folder, count in val_image_count.items():
    print(f"{folder}: {count} gambar")
print(f"Total gambar di val_split: {total_val_images} gambar\n")

print("Jumlah gambar di setiap folder test_split:")
for folder, count in test_image_count.items():
    print(f"{folder}: {count} gambar")
print(f"Total gambar di test_split: {total_test_images} gambar\n")

Jumlah gambar di setiap folder train_split:
/content/drive/MyDrive/HOLOGY/dataset/train_split_warna: 0 gambar
/content/drive/MyDrive/HOLOGY/dataset/train_split_warna/Putih: 110 gambar
/content/drive/MyDrive/HOLOGY/dataset/train_split_warna/Hitam: 189 gambar
/content/drive/MyDrive/HOLOGY/dataset/train_split_warna/Merah: 95 gambar
/content/drive/MyDrive/HOLOGY/dataset/train_split_warna/Kuning: 98 gambar
/content/drive/MyDrive/HOLOGY/dataset/train_split_warna/Biru: 129 gambar
Total gambar di train_split: 621 gambar

Jumlah gambar di setiap folder val_split:
/content/drive/MyDrive/HOLOGY/dataset/val_split_warna: 0 gambar
/content/drive/MyDrive/HOLOGY/dataset/val_split_warna/Putih: 15 gambar
/content/drive/MyDrive/HOLOGY/dataset/val_split_warna/Merah: 15 gambar
/content/drive/MyDrive/HOLOGY/dataset/val_split_warna/Biru: 18 gambar
/content/drive/MyDrive/HOLOGY/dataset/val_split_warna/Hitam: 18 gambar
/content/drive/MyDrive/HOLOGY/dataset/val_split_warna/Kuning: 12 gambar
Total gambar di val_

This code counts the number of images in each folder and subfolder of the training, validation, and test datasets.

It defines a function `count_images_in_folder()` that walks through each folder using `os.walk()`, checks for `.jpg` and `.png` files, and counts the number of image files. It returns the count of images in each subfolder and the total image count.

The function is applied to the `train_split_folder`, `val_split_folder`, and `test_split_folder`, and the results (number of images in each subfolder and the total) are printed.

In [ ]:
img_size = (224,224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    fill_mode='nearest'
)

# Validation dan Test tidak memerlukan augmentasi, hanya rescaling
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Membuat generator untuk train dataset
train_generator = train_datagen.flow_from_directory(
    train_split_folder,
    target_size=img_size,  # Sesuaikan dengan ukuran input model Anda
    batch_size=batch_size,
    class_mode='categorical',  # Pilih categorical untuk multi-class classification
    shuffle=True,  # Mengacak data pada setiap epoch
    seed=42  # Seed untuk pengacakan dan augmentasi konsisten
)

# Membuat generator untuk validation dataset
val_generator = val_test_datagen.flow_from_directory(
    val_split_folder,
    target_size=img_size,  # Sesuaikan dengan ukuran input model Anda
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,  # Mengacak data pada setiap epoch
    seed=42  # Seed untuk pengacakan dan augmentasi konsisten
)

# Membuat generator untuk test dataset
test_generator = val_test_datagen.flow_from_directory(
    test_split_folder,
    target_size=img_size,  # Sesuaikan dengan ukuran input model Anda
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Biasanya, test data tidak di-shuffle
)

print("Train, validation, dan test generator berhasil dibuat.")

Found 621 images belonging to 5 classes.
Found 78 images belonging to 5 classes.
Found 78 images belonging to 5 classes.
Train, validation, dan test generator berhasil dibuat.


This code sets up image data generators for training, validation, and testing.

It defines image size as 224x224 pixels and a batch size of 32. The `train_datagen` applies data augmentation (with `fill_mode='nearest'`) and rescales the image pixel values to the range of 0-1. For validation and test sets, `val_test_datagen` only rescales the images without augmentation.

The `train_generator`, `val_generator`, and `test_generator` are created using `flow_from_directory()`, which loads images from the respective folders, resizes them, applies augmentation (for training), and organizes them into batches. The training and validation sets are shuffled, while the test set is not, to ensure that it remains in the same order. Finally, a message is printed to confirm the successful creation of the generators.

## Training Model: Warna

### MobileNetV2

In [ ]:
input_shape = (224, 224, 3)
base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')
base_model.trainable = False
model = models.Sequential()
model.add(layers.InputLayer(input_shape=input_shape))
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_3           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 512)                 │         655,872 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 5)                   │           2,565 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,916,421 (11.13 MB)

 Trainable params: 658,437 (2.51 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
# Callbacks untuk penyimpanan model terbaik dan early stopping
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("best_transfer_learning_model.keras", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [ ]:
# Training model menggunakan generator
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,  # Jumlah epoch dapat disesuaikan
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    callbacks=[checkpoint_cb, early_stopping_cb]
)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


19/19 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: 0.3308 - loss: 1.6551 - val_accuracy: 0.4531 - val_loss: 1.2979
Epoch 2/20
 1/19 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.5312 - loss: 1.1963

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 102ms/step - accuracy: 0.5312 - loss: 1.1963 - val_accuracy: 0.5000 - val_loss: 1.3327
Epoch 3/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - accuracy: 0.5660 - loss: 1.1517 - val_accuracy: 0.6250 - val_loss: 1.1244
Epoch 4/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 0.6875 - loss: 0.8442 - val_accuracy: 0.8571 - val_loss: 0.8169
Epoch 5/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.7127 - loss: 0.8958 - val_accuracy: 0.7500 - val_loss: 0.8835
Epoch 6/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step - accuracy: 0.8438 - loss: 0.6916 - val_accuracy: 0.6429 - val_loss: 1.1201
Epoch 7/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 32s 2s/step - accuracy: 0.8403 - loss: 0.6895 - val_accuracy: 0.7188 - val_loss: 0.8262
Epoch 8/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 65ms/step - accuracy: 0.9375 - loss: 0.4645 - val_accuracy: 0.7857 - val_loss: 0.9321
Epoch 9/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - accuracy: 0.8650 - loss: 0.5651 - val_accuracy: 0.7500 - val_loss: 0.727

In [ ]:
# Evaluasi pada test set
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test Accuracy: {test_acc:.2f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.8021 - loss: 0.6494
Test Accuracy: 0.80


### MobileNetV3Large

In [ ]:
# Definisikan input shape
input_shape = (224, 224, 3)

# Menggunakan MobileNetV3Large sebagai base model
base_model = MobileNetV3Large(input_shape=input_shape, include_top=False, weights='imagenet')
base_model.trainable = False  # Kunci semua layer untuk fase pertama

# Bangun model
model = models.Sequential([
    layers.InputLayer(input_shape=input_shape),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dense(5, activation='softmax')  # Sesuaikan dengan jumlah kelas
])

# Kompilasi model
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

model.summary()

# Callbacks untuk penyimpanan model terbaik dan early stopping
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("best_mobilenetv3_model.keras", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

# Training model menggunakan generator
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,  # Jumlah epoch dapat disesuaikan
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    callbacks=[checkpoint_cb, early_stopping_cb]
)

12683000/12683000 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ MobileNetV3Large (Functional)        │ (None, 7, 7, 960)           │       2,996,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_3           │ (None, 960)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 512)                 │         492,032 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 5)                   │           2,565 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,490,949 (13.32 MB)

 Trainable params: 494,597 (1.89 MB)

 Non-trainable params: 2,996,352 (11.43 MB)

Epoch 1/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.2310 - loss: 1.6137 - val_accuracy: 0.2656 - val_loss: 1.5518
Epoch 2/20
 1/19 ━━━━━━━━━━━━━━━━━━━━ 15s 836ms/step - accuracy: 0.4375 - loss: 1.4464

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 100ms/step - accuracy: 0.4375 - loss: 1.4464 - val_accuracy: 0.2143 - val_loss: 1.5920
Epoch 3/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - accuracy: 0.3152 - loss: 1.5050 - val_accuracy: 0.2969 - val_loss: 1.5003
Epoch 4/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 110ms/step - accuracy: 0.4688 - loss: 1.3845 - val_accuracy: 0.2143 - val_loss: 1.5928
Epoch 5/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - accuracy: 0.3748 - loss: 1.4396 - val_accuracy: 0.3750 - val_loss: 1.4789
Epoch 6/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.3438 - loss: 1.4754 - val_accuracy: 0.2857 - val_loss: 1.4799
Epoch 7/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.4043 - loss: 1.4089 - val_accuracy: 0.3906 - val_loss: 1.4288
Epoch 8/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - accuracy: 0.5000 - loss: 1.3496 - val_accuracy: 0.3571 - val_loss: 1.5709
Epoch 9/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - accuracy: 0.4590 - loss: 1.3754 - val_accuracy: 0.4531 - val_loss: 1.39

In [ ]:
# Evaluasi pada test set
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test Accuracy: {test_acc:.2f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step - accuracy: 0.3958 - loss: 1.2656
Test Accuracy: 0.36


### InceptionV3

In [ ]:
# Definisikan input shape
input_shape = (224, 224, 3)

# Menggunakan MobileNetV3Large sebagai base model
base_model = InceptionV3(input_shape=input_shape, include_top=False, weights='imagenet')
base_model.trainable = False  # Kunci semua layer untuk fase pertama

# Bangun model
model = models.Sequential([
    layers.InputLayer(input_shape=input_shape),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dense(5, activation='softmax')  # Sesuaikan dengan jumlah kelas
])

# Kompilasi model
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

model.summary()

# Callbacks untuk penyimpanan model terbaik dan early stopping
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("best_mobilenetv3_model.keras", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

# Training model menggunakan generator
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,  # Jumlah epoch dapat disesuaikan
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    callbacks=[checkpoint_cb, early_stopping_cb]
)

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ inception_v3 (Functional)            │ (None, 5, 5, 2048)          │      21,802,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_4           │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 512)                 │       1,049,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 5)                   │           2,565 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 22,854,437 (87.18 MB)

 Trainable params: 1,051,653 (4.01 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

Epoch 1/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 110s 5s/step - accuracy: 0.3021 - loss: 1.6248 - val_accuracy: 0.5156 - val_loss: 1.2323
Epoch 2/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 233ms/step - accuracy: 0.4062 - loss: 1.3936 - val_accuracy: 0.3571 - val_loss: 1.4500
Epoch 3/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 84s 4s/step - accuracy: 0.6157 - loss: 1.0749 - val_accuracy: 0.6250 - val_loss: 1.0837
Epoch 4/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 7s 191ms/step - accuracy: 0.6562 - loss: 0.9117 - val_accuracy: 0.6429 - val_loss: 1.0369
Epoch 5/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 136s 4s/step - accuracy: 0.7551 - loss: 0.7796 - val_accuracy: 0.6719 - val_loss: 0.9369
Epoch 6/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 80ms/step - accuracy: 0.8125 - loss: 0.6550 - val_accuracy: 0.4286 - val_loss: 1.0760
Epoch 7/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 131s 4s/step - accuracy: 0.8335 - loss: 0.6135 - val_accuracy: 0.6406 - val_loss: 0.9407
Epoch 8/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 15s 688ms/step - accuracy: 0.6562 - loss: 0.7677 - val_accuracy: 0.6429 

In [ ]:
# Evaluasi pada test set
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test Accuracy: {test_acc:.2f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 4s/step - accuracy: 0.7292 - loss: 0.6689
Test Accuracy: 0.72


### XCeption

In [ ]:
# Definisikan input shape
input_shape = (224, 224, 3)

# Menggunakan MobileNetV3Large sebagai base model
base_model = Xception(input_shape=input_shape, include_top=False, weights='imagenet')
base_model.trainable = False  # Kunci semua layer untuk fase pertama

# Bangun model
model = models.Sequential([
    layers.InputLayer(input_shape=input_shape),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dense(5, activation='softmax')  # Sesuaikan dengan jumlah kelas
])

# Kompilasi model
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

model.summary()

# Callbacks untuk penyimpanan model terbaik dan early stopping
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("best_mobilenetv3_model.keras", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

# Training model menggunakan generator
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,  # Jumlah epoch dapat disesuaikan
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    callbacks=[checkpoint_cb, early_stopping_cb]
)

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ xception (Functional)                │ (None, 7, 7, 2048)          │      20,861,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_6           │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 512)                 │       1,049,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 5)                   │           2,565 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,913,133 (83.59 MB)

 Trainable params: 1,051,653 (4.01 MB)

 Non-trainable params: 20,861,480 (79.58 MB)

Epoch 1/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 155s 8s/step - accuracy: 0.2981 - loss: 1.5419 - val_accuracy: 0.4844 - val_loss: 1.2503
Epoch 2/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 12s 221ms/step - accuracy: 0.6250 - loss: 1.1279 - val_accuracy: 0.4286 - val_loss: 1.2304
Epoch 3/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 145s 8s/step - accuracy: 0.6397 - loss: 1.0786 - val_accuracy: 0.5938 - val_loss: 1.0207
Epoch 4/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 12s 315ms/step - accuracy: 0.6875 - loss: 0.8496 - val_accuracy: 0.7143 - val_loss: 0.8888
Epoch 5/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 198s 8s/step - accuracy: 0.7435 - loss: 0.8548 - val_accuracy: 0.7812 - val_loss: 0.8831
Epoch 6/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 9s 194ms/step - accuracy: 0.7500 - loss: 0.8332 - val_accuracy: 0.8571 - val_loss: 0.8747
Epoch 7/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 149s 8s/step - accuracy: 0.8368 - loss: 0.6907 - val_accuracy: 0.7500 - val_loss: 0.7569
Epoch 8/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 139ms/step - accuracy: 0.7812 - loss: 0.6053 - val_accuracy: 0.64

In [ ]:
# Evaluasi pada test set
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test Accuracy: {test_acc:.2f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 14s 8s/step - accuracy: 0.7917 - loss: 0.6586
Test Accuracy: 0.80


### VGG16

In [ ]:
input_shape = (224, 224, 3)
base_model = VGG16(input_shape=input_shape, include_top=False, weights='imagenet')
base_model.trainable = False
model = models.Sequential()
model.add(layers.InputLayer(input_shape=input_shape))
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

model.summary()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 7, 7, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_2           │ (None, 512)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 512)                 │         262,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 5)                   │           2,565 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,979,909 (57.14 MB)

 Trainable params: 265,221 (1.01 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
# Callbacks untuk penyimpanan model terbaik dan early stopping
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("best_transfer_learning_model.keras", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [ ]:
# Training model menggunakan generator
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,  # Jumlah epoch dapat disesuaikan
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    callbacks=[checkpoint_cb, early_stopping_cb]
)

Epoch 1/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 372s 19s/step - accuracy: 0.2186 - loss: 1.6806 - val_accuracy: 0.2344 - val_loss: 1.5742
Epoch 2/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 30s 477ms/step - accuracy: 0.3438 - loss: 1.5382 - val_accuracy: 0.3571 - val_loss: 1.5522
Epoch 3/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 414s 20s/step - accuracy: 0.2805 - loss: 1.5341 - val_accuracy: 0.3281 - val_loss: 1.5352
Epoch 4/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 26s 502ms/step - accuracy: 0.5000 - loss: 1.3564 - val_accuracy: 0.2143 - val_loss: 1.5618
Epoch 5/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 416s 20s/step - accuracy: 0.3443 - loss: 1.4663 - val_accuracy: 0.3281 - val_loss: 1.5002
Epoch 6/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.4062 - loss: 1.3993 - val_accuracy: 0.2857 - val_loss: 1.4484
Epoch 7/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 378s 20s/step - accuracy: 0.4298 - loss: 1.4312 - val_accuracy: 0.3906 - val_loss: 1.4491
Epoch 8/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 27s 528ms/step - accuracy: 0.4062 - loss: 1.3878 - val_accuracy: 0

In [ ]:
# Evaluasi pada test set
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test Accuracy: {test_acc:.2f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 37s 19s/step - accuracy: 0.5312 - loss: 1.2028
Test Accuracy: 0.52


### ResNet50

In [ ]:
input_shape = (224, 224, 3)
base_model = ResNet50(input_shape=input_shape, include_top=False, weights='imagenet')
base_model.trainable = False
model = models.Sequential()
model.add(layers.InputLayer(input_shape=input_shape))
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)                │ (None, 7, 7, 2048)          │      23,587,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │       1,049,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 5)                   │           2,565 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 24,639,365 (93.99 MB)

 Trainable params: 1,051,653 (4.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
# Callbacks untuk penyimpanan model terbaik dan early stopping
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("best_transfer_learning_model.keras", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [ ]:
# Training model menggunakan generator
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,  # Jumlah epoch dapat disesuaikan
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    callbacks=[checkpoint_cb, early_stopping_cb]
)

Epoch 1/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 140s 7s/step - accuracy: 0.2027 - loss: 1.9371 - val_accuracy: 0.2031 - val_loss: 1.6478
Epoch 2/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 187ms/step - accuracy: 0.1538 - loss: 1.6739 - val_accuracy: 0.2857 - val_loss: 1.6055
Epoch 3/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 123s 6s/step - accuracy: 0.3802 - loss: 1.5066 - val_accuracy: 0.2656 - val_loss: 1.4964
Epoch 4/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 9s 116ms/step - accuracy: 0.3438 - loss: 1.4210 - val_accuracy: 0.2143 - val_loss: 1.5472
Epoch 5/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 123s 6s/step - accuracy: 0.3741 - loss: 1.4395 - val_accuracy: 0.2344 - val_loss: 1.5428
Epoch 6/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 186ms/step - accuracy: 0.3750 - loss: 1.3806 - val_accuracy: 0.4286 - val_loss: 1.2563
Epoch 7/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 114s 6s/step - accuracy: 0.3964 - loss: 1.3430 - val_accuracy: 0.4375 - val_loss: 1.4205
Epoch 8/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 9s 242ms/step - accuracy: 0.6250 - loss: 1.2562 - val_accuracy: 0.5000

In [ ]:
# Evaluasi pada test set
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test Accuracy: {test_acc:.2f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 5s/step - accuracy: 0.4479 - loss: 1.3343
Test Accuracy: 0.42


### EfficientNetV2B0

In [ ]:
input_shape = (224, 224, 3)
base_model = EfficientNetV2B0(input_shape=input_shape, include_top=False, weights='imagenet')
base_model.trainable = False
model = models.Sequential()
model.add(layers.InputLayer(input_shape=input_shape))
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

model.summary()

24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetv2-b0 (Functional)       │ (None, 7, 7, 1280)          │       5,919,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │         655,872 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │           2,565 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,577,749 (25.09 MB)

 Trainable params: 658,437 (2.51 MB)

 Non-trainable params: 5,919,312 (22.58 MB)

In [ ]:
# Callbacks untuk penyimpanan model terbaik dan early stopping
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("best_transfer_learning_model.keras", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

In [ ]:
# Training model menggunakan generator
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,  # Jumlah epoch dapat disesuaikan
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    callbacks=[checkpoint_cb, early_stopping_cb]
)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


19/19 ━━━━━━━━━━━━━━━━━━━━ 232s 9s/step - accuracy: 0.3016 - loss: 1.5967 - val_accuracy: 0.3594 - val_loss: 1.6028
Epoch 2/20
 1/19 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.1875 - loss: 1.5681

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 144ms/step - accuracy: 0.1875 - loss: 1.5681 - val_accuracy: 0.2143 - val_loss: 1.6367
Epoch 3/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.2082 - loss: 1.6063 - val_accuracy: 0.2188 - val_loss: 1.6311
Epoch 4/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 4s 195ms/step - accuracy: 0.3077 - loss: 1.5978 - val_accuracy: 0.2143 - val_loss: 1.6218
Epoch 5/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.3231 - loss: 1.5609 - val_accuracy: 0.2812 - val_loss: 1.5757
Epoch 6/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - accuracy: 0.3125 - loss: 1.5746 - val_accuracy: 0.1429 - val_loss: 1.6661
Epoch 7/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.3085 - loss: 1.5707 - val_accuracy: 0.2344 - val_loss: 1.6288
Epoch 8/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.2500 - loss: 1.6092 - val_accuracy: 0.2857 - val_loss: 1.6318
Epoch 9/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.3123 - loss: 1.5593 - val_accuracy: 0.2031 - val_loss: 1.58

In [ ]:
# Evaluasi pada test set
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test Accuracy: {test_acc:.2f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - accuracy: 0.4583 - loss: 1.4898
Test Accuracy: 0.42


### DenseNet

In [ ]:
# Definisikan input shape
input_shape = (224, 224, 3)

# Menggunakan MobileNetV3Large sebagai base model
base_model = DenseNet121(input_shape=input_shape, include_top=False, weights='imagenet')
base_model.trainable = False  # Kunci semua layer untuk fase pertama

# Bangun model
model = models.Sequential([
    layers.InputLayer(input_shape=input_shape),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dense(5, activation='softmax')  # Sesuaikan dengan jumlah kelas
])

# Kompilasi model
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

model.summary()

# Callbacks untuk penyimpanan model terbaik dan early stopping
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("best_mobilenetv3_model.keras", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

# Training model menggunakan generator
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,  # Jumlah epoch dapat disesuaikan
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    callbacks=[checkpoint_cb, early_stopping_cb]
)

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ densenet121 (Functional)             │ (None, 7, 7, 1024)          │       7,037,504 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_5           │ (None, 1024)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 5)                   │           2,565 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,564,869 (28.86 MB)

 Trainable params: 527,365 (2.01 MB)

 Non-trainable params: 7,037,504 (26.85 MB)

Epoch 1/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 135s 6s/step - accuracy: 0.2269 - loss: 1.7252 - val_accuracy: 0.2656 - val_loss: 1.5390
Epoch 2/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 175ms/step - accuracy: 0.3125 - loss: 1.4751 - val_accuracy: 0.3571 - val_loss: 1.5068
Epoch 3/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 127s 6s/step - accuracy: 0.4790 - loss: 1.2670 - val_accuracy: 0.5312 - val_loss: 1.2661
Epoch 4/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 207ms/step - accuracy: 0.6250 - loss: 1.3621 - val_accuracy: 0.4286 - val_loss: 1.3728
Epoch 5/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 135s 6s/step - accuracy: 0.6866 - loss: 1.0420 - val_accuracy: 0.6094 - val_loss: 1.0748
Epoch 6/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 28s 1s/step - accuracy: 0.7500 - loss: 0.9614 - val_accuracy: 0.7143 - val_loss: 0.9347
Epoch 7/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 110s 6s/step - accuracy: 0.7174 - loss: 0.9227 - val_accuracy: 0.7500 - val_loss: 0.9656
Epoch 8/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 8s 180ms/step - accuracy: 0.7500 - loss: 0.8351 - val_accuracy: 0.6429 -

In [ ]:
# Evaluasi pada test set
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test Accuracy: {test_acc:.2f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 4s/step - accuracy: 0.7812 - loss: 0.6491
Test Accuracy: 0.81


### DenseNet Unfreeze

In [ ]:
# Definisikan input shape
input_shape = (224, 224, 3)

# Menggunakan DenseNet121 sebagai base model
base_model = DenseNet121(input_shape=input_shape, include_top=False, weights='imagenet')
base_model.trainable = True  # Setel semua layer menjadi trainable

# Unfreeze beberapa layer di bagian atas
# Anda bisa menentukan berapa banyak layer yang ingin di-unfreeze
for layer in base_model.layers[-20:]:  # Unfreeze 20 layer terakhir
    layer.trainable = True

# Bangun model
model = models.Sequential([
    layers.InputLayer(input_shape=input_shape),
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dense(5, activation='softmax')  # Sesuaikan dengan jumlah kelas
])

# Kompilasi model
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

model.summary()

# Callbacks untuk penyimpanan model terbaik dan early stopping
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("best_densenet_model.keras", save_best_only=True)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

# Training model menggunakan generator
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=20,  # Jumlah epoch dapat disesuaikan
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    callbacks=[checkpoint_cb, early_stopping_cb]
)

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ densenet121 (Functional)             │ (None, 7, 7, 1024)          │       7,037,504 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1024)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │           2,565 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,564,869 (28.86 MB)

...

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


19/19 ━━━━━━━━━━━━━━━━━━━━ 752s 32s/step - accuracy: 0.4996 - loss: 1.2079 - val_accuracy: 0.6875 - val_loss: 0.9674
Epoch 2/20
 1/19 ━━━━━━━━━━━━━━━━━━━━ 2:21 8s/step - accuracy: 1.0000 - loss: 0.2609

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


19/19 ━━━━━━━━━━━━━━━━━━━━ 12s 253ms/step - accuracy: 1.0000 - loss: 0.2609 - val_accuracy: 0.5000 - val_loss: 1.0434
Epoch 3/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 488s 26s/step - accuracy: 0.9906 - loss: 0.0842 - val_accuracy: 0.8594 - val_loss: 0.4236
Epoch 4/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 66s 2s/step - accuracy: 1.0000 - loss: 0.0091 - val_accuracy: 0.7857 - val_loss: 0.5854
Epoch 5/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 493s 25s/step - accuracy: 0.9953 - loss: 0.0203 - val_accuracy: 0.9688 - val_loss: 0.2030
Epoch 6/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 30s 231ms/step - accuracy: 1.0000 - loss: 0.0084 - val_accuracy: 1.0000 - val_loss: 0.1286
Epoch 7/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 531s 25s/step - accuracy: 0.9976 - loss: 0.0159 - val_accuracy: 1.0000 - val_loss: 0.1051
Epoch 8/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 29s 156ms/step - accuracy: 1.0000 - loss: 0.0058 - val_accuracy: 1.0000 - val_loss: 0.1086
Epoch 9/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 541s 26s/step - accuracy: 0.9955 - loss: 0.0141 - val_accuracy: 0.9531 - val

In [ ]:
# Evaluasi pada test set
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f"Test Accuracy: {test_acc:.2f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 9s 4s/step - accuracy: 0.9375 - loss: 0.1839
Test Accuracy: 0.95


In [ ]:
# Simpan model setelah pelatihan
model.save("/content/drive/MyDrive/HOLOGY/final_densenet_model.keras")
print("Model disimpan sebagai 'final_densenet_model.keras'")

Model disimpan sebagai 'final_densenet_model.keras'


This model uses the **DenseNet121** base model with pre-trained weights from ImageNet. By unfreezing the last 20 layers, it allows the model to fine-tune these layers based on the specific dataset, improving its ability to learn from the data while retaining useful features learned during pre-training.

The combination of fine-tuning and the use of a strong pre-trained model like DenseNet121 helps achieve higher accuracy compared to training a model from scratch. The model also uses callbacks like **ModelCheckpoint** (to save the best model) and **EarlyStopping** (to prevent overfitting by stopping if validation accuracy doesn’t improve for a set number of epochs), making it more efficient in training. This approach logically leads to a better-performing model, as it can leverage prior knowledge while learning from the new data.

## Inference Model: Warna

In [ ]:
# Path folder test
test_folder = "/content/drive/MyDrive/HOLOGY/dataset/test"

# Membuat list untuk menyimpan hasil prediksi warna
warna_predictions = []  # Untuk menyimpan prediksi warna (0 hingga 4)

# Mapping warna hasil prediksi
warna_mapping = {0: 'Merah', 1: 'Kuning', 2: 'Biru', 3: 'Hitam', 4: 'Putih'}

# Looping untuk memproses gambar dari 778 hingga 1111 secara berurutan
for img_id in final_df['id']:  # Menggunakan 'id' dari final_df yang sudah ada
    # Cek apakah file .jpg atau .png ada
    img_jpg = os.path.join(test_folder, f"{img_id}.jpg")
    img_png = os.path.join(test_folder, f"{img_id}.png")

    if os.path.exists(img_jpg):
        img_path = img_jpg
    elif os.path.exists(img_png):
        img_path = img_png
    else:
        # Jika tidak ditemukan, lanjutkan ke gambar berikutnya
        print(f"Gambar dengan id {img_id} tidak ditemukan.")
        continue

    # Load dan preproses gambar
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Tambahkan dimensi batch
    img_array = img_array / 255.0  # Normalisasi

    # Prediksi menggunakan model warna yang telah dilatih
    warna_pred = model.predict(img_array)  # Gunakan model untuk prediksi warna
    warna_class = np.argmax(warna_pred, axis=1)[0]  # Mendapatkan indeks kelas (0 hingga 4)

    # Simpan warna berdasarkan prediksi model
    warna_predictions.append(warna_mapping[warna_class])

# Menambahkan kolom warna ke final_df
final_df['warna'] = warna_predictions

# Menampilkan DataFrame setelah perubahan
print(final_df[['id', 'jenis', 'warna']].head(20))

     id  jenis   warna
0   778      0   Hitam
1   779      1   Hitam
2   780      1    Biru
3   781      0  Kuning
4   782      1   Hitam
5   783      0   Hitam
6   784      1  Kuning
7   785      1   Hitam
8   786      1    Biru
9   787      0   Hitam
10  788      0   Hitam
11  789      0   Putih
12  790      0    Biru
13  791      0    Biru
14  792      0   Hitam
15  793      1   Merah
16  794      0  Kuning
17  795      0   Merah
18  796      0  Kuning
19  797      0    Biru


This code processes images from the test folder using the image IDs from `final_df`. For each image, it checks if the `.jpg` or `.png` file exists, then loads and preprocesses the image. The preprocessed image is fed into the trained color prediction model to predict its color. The predicted color (mapped from numerical labels 0-4) is appended to the `warna_predictions` list. After all predictions are made, the `final_df` DataFrame is updated with a new column for the predicted color (`warna`), and the first 20 rows of the updated DataFrame are printed.

In [ ]:
# Mapping perubahan warna menjadi angka
warna_replace_mapping = {
    "Kuning": 1,
    "Merah": 0,
    "Biru": 2,
    "Hitam": 3,
    "Putih": 4
}

# Mengganti nilai di kolom 'warna' berdasarkan mapping
final_df['warna'] = final_df['warna'].replace(warna_replace_mapping)

# Menampilkan DataFrame setelah perubahan
print(final_df[['id', 'jenis', 'warna']].head(20))

     id  jenis  warna
0   778      0      3
1   779      1      3
2   780      1      2
3   781      0      1
4   782      1      3
5   783      0      3
6   784      1      1
7   785      1      3
8   786      1      2
9   787      0      3
10  788      0      3
11  789      0      4
12  790      0      2
13  791      0      2
14  792      0      3
15  793      1      0
16  794      0      1
17  795      0      0
18  796      0      1
19  797      0      2


<ipython-input-43-083c9b74ddaf>:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df['warna'] = final_df['warna'].replace(warna_replace_mapping)


The code defines a `warna_replace_mapping` dictionary that maps the color names (e.g., "Kuning", "Merah") to corresponding numeric labels (e.g., 1 for "Kuning", 0 for "Merah"). It then replaces the color names in the `final_df['warna']` column with their respective numeric values according to this mapping. This transformation is necessary for submission, as the model outputs numerical labels, and the submission format requires these numeric values instead of the color names.

# Save dataframe for submission

In [ ]:
final_df

,id,jenis,warna
0,778,0,3
1,779,1,3
2,780,1,2
3,781,0,1
4,782,1,3
...,...,...,...
329,1107,1,0
330,1108,0,2
331,1109,0,2
332,1110,1,3


In [ ]:
final_df.to_csv("/content/drive/MyDrive/HOLOGY/dataset/sample_submission_best.csv",index=False)